<a href="https://colab.research.google.com/github/HagarIbrahiem/Generative-Models/blob/main/Generative%20Models%20%7C%20GANs%20%7C%20Images%20of%20Hands%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implments a Generative Adversarial Network (GAN) - DCGAN that generates pictures of hands. These will be trained on a dataset of hand images doing sign language.

## Imports Libs

In [16]:
import tensorflow as tf
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import numpy as np

import urllib.request
import zipfile
from IPython import display

## Utilities

In [17]:
def plot_results(images, n_cols=None):
    '''visualizes fake images'''
    display.clear_output(wait=False)

    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1

    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)

    plt.figure(figsize=(n_cols, n_rows))

    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

## Load Data

In [1]:
# download the dataset
training_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/Resources/signs-training.zip"
training_file_name = "signs-training.zip"
urllib.request.urlretrieve(training_url, training_file_name)

# extract to local directory
training_dir = "/tmp"
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

## Preprocess images

Prepare the dataset to a format suitable for the model:
- read the files,
- convert it to a tensor of floats,
- then normalize the pixel values.

In [ ]:
BATCH_SIZE = 32

# mapping function for preprocessing the image files
def map_images(file):
  '''converts the images to floats and normalizes the pixel values'''
  img = tf.io.decode_png(tf.io.read_file(file))
  img = tf.dtypes.cast(img, tf.float32)
  img = img / 255.0

  return img

# create training batches
filename_dataset = tf.data.Dataset.list_files("/tmp/signs-training/*.png")
image_dataset = filename_dataset.map(map_images).batch(BATCH_SIZE)

## Build Model

### Build Generator

Using the following architecture:
- *Dense*: number of units should equal `7 * 7 * 128`, input_shape takes in a list containing the random normal dimensions.
    - `random_normal_dimensions` is a hyperparameter that defines how many random numbers in a vector you'll want to feed into the generator as a starting point for generating images.
- *Reshape*: reshape the vector to a 7 x 7 x 128 tensor.
- *BatchNormalization*
- *Conv2DTranspose*: takes `64` units, kernel size is `5`, strides is `2`, padding is `SAME`, activation is `selu`.
- *BatchNormalization*
- *Conv2DTranspose*: `1` unit, kernel size is `5`, strides is `2`, padding is `SAME`, and activation is `tanh`.

In [ ]:
# pass the random_normal_dimensions to the first dense layer of the generator
random_normal_dimensions = 32

generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape=[random_normal_dimensions]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME",
                                 activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="SAME",
                                 activation="tanh"),
])

generator.summary()

### Build Discriminator

Using following architecture for the discriminator:
- *Conv2D*: 64 units, kernel size of 5, strides of 2, padding is SAME, activation is a leaky relu with alpha of 0.2, input shape is 28 x 28 x 1
- *Dropout*: rate is 0.4 (fraction of input units to drop)
- *Conv2D*: 128 units, kernel size of 5, strides of 2, padding is SAME, activation is LeakyRelu with alpha of 0.2
- *Dropout*: rate is 0.4.
- *Flatten*
- *Dense*: with 1 unit and a sigmoid activation

In [ ]:
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[28, 28, 1]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])

discriminator.summary()

#### Compile Discriminator

- Compile the discriminator with a binary_crossentropy loss and rmsprop optimizer.
- Set the discriminator to not train on its weights (set its "trainable" field).

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False

## Build and compile the GAN model

- Build the sequential model for the GAN, passing a list containing the generator and discriminator.
- Compile the model with a binary cross entropy loss and rmsprop optimizer.

In [ ]:
gan = keras.models.Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

## Train the GAN

Phase 1
- real_batch_size: Get the batch size of the input batch (it's the zero-th dimension of the tensor)
- noise: Generate the noise using `tf.random.normal`.  The shape is batch size x random_normal_dimension
- fake images: Use the generator that you just created. Pass in the noise and produce fake images.
- mixed_images: concatenate the fake images with the real images.
  - Set the axis to 0.
- discriminator_labels: Set to `0.` for fake images and `1.` for real images.
- Set the discriminator as trainable.
- Use the discriminator's `train_on_batch()` method to train on the mixed images and the discriminator labels.


Phase 2
- noise: generate random normal values with dimensions batch_size x random_normal_dimensions
  - Use `real_batch_size`.
- Generator_labels: Set to `1.` to mark the fake images as real
  - The generator will generate fake images that are labeled as real images and attempt to fool the discriminator.
- Set the discriminator to NOT be trainable.
- Train the GAN on the noise and the generator labels.

In [ ]:
def train_gan(gan, dataset, random_normal_dimensions, n_epochs=50):
    """ Defines the two-phase training loop of the GAN
    Args:
      gan -- the GAN model which has the generator and discriminator
      dataset -- the training set of real images
      random_normal_dimensions -- dimensionality of the input to the generator
      n_epochs -- number of epochs
    """

    # get the two sub networks from the GAN model
    generator, discriminator = gan.layers

    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for real_images in dataset:

            # infer batch size from the current batch of real images
            real_batch_size = real_images.shape[0]

            # Train the discriminator - PHASE 1
            # Create the noise
            noise = tf.random.normal(shape=[real_batch_size, random_normal_dimensions])


            # Use the noise to generate fake images
            fake_images = generator(noise)

            # Create a list by concatenating the fake images with the real ones
            mixed_images = tf.concat([fake_images, real_images], axis=0)


            # Create the labels for the discriminator
            # 0 for the fake images
            # 1 for the real images
            discriminator_labels = tf.constant([[0.]] * real_batch_size + [[1.]] * real_batch_size)


            # Ensure that the discriminator is trainable
            discriminator.trainable = True

            # Use train_on_batch to train the discriminator with the mixed images and the discriminator labels
            discriminator.train_on_batch(mixed_images, discriminator_labels)

            # Train the generator - PHASE 2
            # create a batch of noise input to feed to the GAN
            noise = tf.random.normal(shape=[real_batch_size, random_normal_dimensions])

            # label all generated images to be "real"
            generator_labels = tf.constant([[1.]] * real_batch_size)

            # Freeze the discriminator
            discriminator.trainable=False

            # Train the GAN on the noise with the labels all set to be true
            gan.train_on_batch(noise, generator_labels)

        plot_results(fake_images, 16)
        plt.show()
    return fake_images

## Run the training
For each epoch, a set of 31 images will be displayed onscreen. The longer the  train, the better the output fake images will be.

In [ ]:
# we can adjust the number of epochs
EPOCHS = 60

# run the training loop and collect images
fake_images = train_gan(gan, image_dataset, random_normal_dimensions, EPOCHS)